# Netzwerkvisualisierung mit Python
In diesem Beispiel bilden wir Netzwerke ab, die auf der Nennung von Personen im jeweils gleichen Brief beruhen.

Um eine Netzwerkvisualisierung zu erstellen, wollen wir Daten in einem quasi-tabellarischen Format:
- Knoten A
- Knoten B

Weitere Infos könnten sein, wie häufig zwei Knoten verbunden sind, von welcher Art die Verbindung ist, etc.

In [ ]:
# import der relevanten Bibliotheken
from lxml import etree as et
import glob
import re

In [ ]:
# für jeden Brief, halte alle Verbindungen im Brief fest
import itertools
from collections import Counter

all_pairs = []

for letter in glob.glob("docs/*.xml"):
    tree = et.parse(letter)
    root = tree.getroot()

    person_keys = root.xpath("./tei:text/tei:body/tei:div[@type='content']//tei:persName/@key", namespaces={"tei":"http://www.tei-c.org/ns/1.0"})

    unique_keys = set(person_keys)
    
    # indem wir die Liste sortieren stellen wir sicher, dass unsere Paare ungerichtet gezählt werden können
    person_keys = sorted(unique_keys)
    
    pairs = list(itertools.combinations(person_keys, 2))
    
    all_pairs.extend(pairs)

counted_pairs = Counter(all_pairs)

print(counted_pairs.most_common())

Wir haben nun alle Kanten samt ihrer Häufigkeit. Wir könnten diese nun abspeichern und in einem externen Programm, z.B. Palladio oder Gephi weiterverarbeiten. Aber in dem Fall machen wir direkt in Python weiter! Mit [*pyvis*](https://pyvis.readthedocs.io/en/latest/) können interaktive Graphen generiert werden, die man sich dann im Browser ansehen kann.

In [ ]:
# Installiere Pyvis
%pip install -U pyvis

In [ ]:
from pyvis.network import Network

# Initialisiere ein Netzwerk-Objekt
net = Network(notebook=True, cdn_resources="remote")

# Füge unsere Kanten samt Gewichten dem Netzwerk hinzu
for pair, weight in counted_pairs.most_common():
    # Stelle sicher, dass die Knoten existieren
    for node in pair:
        net.add_node(node, title=node)
    
    # Füge die Kanten hinzu mit der Häufigkeit als Gewichtung
    net.add_edge(pair[0], pair[1], value=weight)

net.force_atlas_2based()  # Erstelle den Graph mit einem Physik-System
net.show_buttons(filter_=["physics"])  # Zeige ein UI an, mit dem man das System anpassen kann

# Schreibe eine HTML-Datei, die man im Browser öffnen kann.
net.show("interactive_network.html")


Nun sind leider die Personen nur durch ihren Key wiedergegeben, aber vielleicht können wir das wettmachen, in dem wir implementieren, dass ein Click auf einen Knoten die Seite der Person im Register öffnet!

In [22]:
from pyvis.network import Network

# Initialisiere ein Netzwerk-Objekt
net = Network(notebook=True, cdn_resources="remote")

# Füge unsere Kanten samt Gewichten dem Netzwerk hinzu
for pair, weight in counted_pairs.most_common():
    # Stelle sicher, dass die Knoten existieren
    for node in pair:
        net.add_node(node, title=node, url="https://sturm-edition.de/register/personen/" + node + ".html")
    
    # Füge die Kanten hinzu mit der Häufigkeit als Gewichtung
    net.add_edge(pair[0], pair[1], value=weight)

net.force_atlas_2based()  # Erstelle den Graph mit einem Physik-System
net.show_buttons(filter_=["physics"])  # Zeige ein UI an, mit dem man das System anpassen kann

# Schreibe eine HTML-Datei, die man im Browser öffnen kann.
net.show("interactive_network.html")


interactive_network.html
